In [1]:
import itertools
import collections
import copy
from functools import cache

In [2]:
# from https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/
import heapq


def calculate_distances(graph, starting_vertex):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Only consider this new path if it's better than any path we've
            # already found.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))

    return distances

## part 2 ##

    e---f---g---h---i---j---k (hallway)
         \ / \ / \ / \ / 
          a3  b3  c3  d3 (rooms)
          |   |   |   |
          a2  b2  c2  d2
          |   |   |   |
          a1  b1  c1  d1
          |   |   |   |
          a0  bO  cO  dO
          
Weights are 1 for e-f, j-k, in between rooms (a0-a2-a3-a1, etc), and 2 for all others.
The weights of 2 make it so we don't have to have rules about not stopping on a hallway 
space above one of the rooms.


In [3]:
graph  = {'a0': {'a1': 1},
          'b0': {'b1': 1},
          'c0': {'c1': 1},
          'd0': {'d1': 1},
          'a1': {'a0': 1, 'a2': 1},
          'b1': {'b0': 1, 'b2': 1},
          'c1': {'c0': 1, 'c2': 1},
          'd1': {'d0': 1, 'd2': 1},
          'a2': {'a1': 1, 'a3': 1},
          'b2': {'b1': 1, 'b3': 1},
          'c2': {'c1': 1, 'c3': 1},
          'd2': {'d1': 1, 'd3': 1},
          'a3': {'a2': 1, 'f': 2, 'g': 2},
          'b3': {'b2': 1, 'g': 2, 'h': 2},
          'c3': {'c2': 1, 'h': 2, 'i': 2},
          'd3': {'d2': 1, 'i': 2, 'j': 2},
          'e': {'f': 1},
          'f': {'e': 1, 'a3': 2, 'g': 2},
          'g': {'f': 2, 'h': 2, 'a3': 2, 'b3': 2},
          'h': {'g': 2, 'i': 2, 'b3': 2, 'c3': 2},
          'i': {'h': 2, 'j': 2, 'c3': 2, 'd3': 2},
          'j': {'i': 2, 'k': 1, 'd3': 2},
          'k': {'j': 1},
        }
nodes = graph.keys()
nodeidx = {val:i for i,val in enumerate(nodes)}
hallway = ['e', 'f', 'g', 'h', 'i', 'j', 'k']
rooms  = [''.join(p) for p in itertools.product('abcd','0123')]
move_cost = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [4]:
example_start = {'a3': 'B', 'b3': 'C', 'c3': 'B', 'd3': 'D',
                 'a2': 'D', 'b2': 'C', 'c2': 'B', 'd2': 'A',
                 'a1': 'D', 'b1': 'B', 'c1': 'A', 'd1': 'C',
                 'a0': 'A', 'b0': 'D', 'c0': 'C', 'd0': 'A'}
puzzle_start =  {'a3': 'D', 'b3': 'B', 'c3': 'C', 'd3': 'A',
                 'a2': 'D', 'b2': 'C', 'c2': 'B', 'd2': 'A',
                 'a1': 'D', 'b1': 'B', 'c1': 'A', 'd1': 'C',
                 'a0': 'C', 'b0': 'A', 'c0': 'D', 'd0': 'B'}
Positions = collections.namedtuple('Positions', nodes)
for node in hallway:
    example_start[node] = None
    puzzle_start[node] = None
example_init = Positions(*[example_start[node] for node in nodes])
puzzle_init = Positions(*[puzzle_start[node] for node in nodes])

In [5]:
def is_finished(pos):
    for node in rooms:
        col = node[0]
        val = pos[nodeidx[node]]
        if (val is None) or (val.lower() != col):
            return False
    return True

In [6]:
finished_ex = Positions(*itertools.chain(*itertools.repeat('ABCD', 4), itertools.repeat(None, len(hallway))))
is_finished(finished_ex), is_finished(example_init), is_finished(puzzle_init)

(True, False, False)

In [7]:
def find_valid_rooms(pos):
    valid = []
    for col in 'abcd':
        empty = None
        for row in reversed(range(4)):
            loc = f'{col}{row}'
            val = pos[nodeidx[loc]]
            if val is None:
                empty = row
        if empty == 0:
            valid.append(f'{col}{empty}')
            continue
        if empty is None:
            continue
        if all(pos[nodeidx[f'{col}{row}']].lower() == col for row in reversed(range(empty))):
            valid.append(f'{col}{empty}')
    return valid

In [8]:
valid_test = Positions(a0='B', b0='B', c0='C', d0=None, a1=None, b1='B', c1='C', d1=None, a2=None, b2='B', c2='C', d2=None, a3=None, b3='B', c3=None, d3=None, e=None, f=None, g=None, h=None, i=None, j=None, k=None)

In [9]:
find_valid_rooms(valid_test), find_valid_rooms(example_init)

(['c3', 'd0'], [])

In [10]:
@cache
def find_topmost_moveable(pos):
    can_move = []
    for col in 'abcd':
        for row in reversed(range(4)):
            loc = f'{col}{row}'
            val = pos[nodeidx[loc]]
            if val:
                if any(pos[nodeidx[f'{col}{r}']].lower() != col for r in reversed(range(row+1))):
                    can_move.append(loc)
                break
    return can_move

In [11]:
find_topmost_moveable(valid_test), find_topmost_moveable(finished_ex), find_topmost_moveable(example_init)

(['a0'], [], ['a3', 'b3', 'c3', 'd3'])

In [12]:
@cache
def traversal_costs(pos, startnode):
    newgraph = copy.deepcopy(graph)
    for node in graph:
        for endpt in graph[node]:
            if pos[nodeidx[endpt]] is not None:
                newgraph[node][endpt] = float('infinity')
    return calculate_distances(newgraph, startnode)

In [13]:
def allowed_moves(pos, currcost):
    topmost = find_topmost_moveable(pos)
    hallway_occ = [node for node in hallway if pos[nodeidx[node]] is not None]
    # see if anything can move into it's final position
    end_rooms = find_valid_rooms(pos)
    for end_room in end_rooms:
        col = end_room[0]
        for loc in (topmost + hallway_occ):
            val = pos[nodeidx[loc]]
            home_col = val.lower()
            if home_col != col:
                continue
            tcosts = traversal_costs(pos, loc)
            tcost = tcosts[end_room]
            if tcost < float('infinity'):
                # can move to home room
                newpos = list(pos)
                newpos[nodeidx[end_room]] = val
                newpos[nodeidx[loc]] = None
                cost = tcost*move_cost[val] + currcost
                yield Positions(*newpos), cost
                # don't generate any other alternatives, just do this move
                return
    # no moves to home, so generate all possible moves from the rooms into the hallway
    hallway_empty = [node for node in hallway if pos[nodeidx[node]] is None]
    for toprm, hall in itertools.product(topmost, hallway_empty):
        tcosts = traversal_costs(pos, toprm)
        tcost = tcosts[hall]
        if tcost < float('infinity'):
            # can make the move
            val = pos[nodeidx[toprm]]
            newpos = list(pos)
            newpos[nodeidx[hall]] = val
            newpos[nodeidx[toprm]] = None
            cost = tcost*move_cost[val] + currcost
            yield Positions(*newpos), cost

In [14]:
def solve(startpos):
    curr_costs = {startpos: 0}
    curr_positions = [startpos]
    finished_costs = []
    while curr_positions:
        new_positions = set()
        for pos in curr_positions:
            currcost = curr_costs[pos]
            for allowedpos, cost in allowed_moves(pos, currcost):
                if is_finished(allowedpos):
                    finished_costs.append(cost)
                    continue
                if allowedpos in curr_costs:
                    if cost < curr_costs[allowedpos]:
                        curr_costs[allowedpos] = cost
                        new_positions.add(allowedpos)
                else:
                    curr_costs[allowedpos] = cost
                    new_positions.add(allowedpos)
        curr_positions = new_positions
        print(len(curr_positions))
    print('min cost = ', min(finished_costs))
                        

In [15]:
solve(example_init)

28
331
2202
8527
18830
22350
12813
4200
2860
1997
1430
1008
802
574
359
175
70
37
17
10
7
2
0
min cost =  44169


In [16]:
solve(puzzle_init)

28
355
2487
10250
24337
30845
17511
3822
3144
2105
1215
505
151
78
35
20
8
7
6
5
5
5
4
3
2
0
min cost =  47064
